In [5]:
from dotenv import load_dotenv
import os
from qiskit import *
from qiskit_ibm_provider import IBMProvider
from math import *
import qiskit
from qiskit.circuit.library.standard_gates import *
load_dotenv()
IBM_KEY = os.getenv("API_KEY")
provider = IBMProvider()

## CHSH Game
Three party game: Alice, Charlie and Bob  
Charlie selects 2 bits $x$, $y$ uniformly and independently and gives them to Alice and Bob who cannot communicate with each other.  
Alice and Bob must output $a$ and $b$ respectively such that, $$x\cdot y = a \oplus b$$  
In a classical setting, winning for Alice and Bob is possible with a probability of $0.75$ in the optimal case but if they share an entagled bit beforehand, this increases to $0.854$

In [108]:
qin = QuantumRegister(2)
qout = QuantumRegister(2)
cin = ClassicalRegister(2)
cout = ClassicalRegister(2)
qc = QuantumCircuit(qin, qout, cin, cout)

# Pre-game preparation of entanglement
qc.h(qout[0])
qc.cx(qout[0], qout[1])
# Alice gets access to qout[0] and Bob gets access to qout[1]
qc.barrier()

# Charlie obtains x,y uniformly and independently at random
qc.h([qin[0],qin [1]])
qc.measure(qin[0], cin[1]) #This is x
qc.measure(qin[1], cin[0]) #This is y
qc.barrier()

# Application of controlled U3 gate based on x result
U_Alice = UGate(pi/2, 0, pi).control(1)
qc.append(U_Alice, [qin[0], qout[0]])
qc.measure(qout[0], cout[1])

#Depending on the state of y Bob performs his operations.
#We put X-gate because we want the below gate to be applied only if qin =0. So we use X(qin) as control.
qc.x(qin[1])
U_Bob1 = UGate(-pi/4, 0, 0).control(1)
qc.append(U_Bob1, [qin[1], qout[1]])

qc.x(qin[1])
U_Bob2 = UGate(pi/4, 0, 0).control(1)
qc.append(U_Bob2, [qin[1], qout[1]])
qc.measure(qout[1], cout [0])

qc.draw()

░ ┌───┐┌─┐    ░                                            »
q600_0: ───────────░─┤ H ├┤M├────░────────■───────────────────────────────────»
                   ░ ├───┤└╥┘┌─┐ ░        │        ┌───┐                 ┌───┐»
q600_1: ───────────░─┤ H ├─╫─┤M├─░────────┼────────┤ X ├────────■────────┤ X ├»
        ┌───┐      ░ └───┘ ║ └╥┘ ░ ┌──────┴───────┐└┬─┬┘        │        └───┘»
q601_0: ┤ H ├──■───░───────╫──╫──░─┤ U(π/2,0,π,0) ├─┤M├─────────┼─────────────»
        └───┘┌─┴─┐ ░       ║  ║  ░ └──────────────┘ └╥┘ ┌───────┴───────┐     »
q601_1: ─────┤ X ├─░───────╫──╫──░───────────────────╫──┤ U(-π/4,0,0,0) ├─────»
             └───┘ ░       ║  ║  ░                   ║  └───────────────┘     »
 c12: 2/═══════════════════╩══╩══════════════════════╬════════════════════════»
                           1  0                      ║                        »
 c13: 2/═════════════════════════════════════════════╩════════════════════════»
                                                     1                        »
«                           
«q600_0: ───────────────────
«                           
«q600_1: ───────■───────────
«               │           
«q601_0: ───────┼───────────
«        ┌──────┴───────┐┌─┐
«q601_1: ┤ U(π/4,0,0,0) ├┤M├
«        └──────────────┘└╥┘
« c12: 2/═════════════════╬═
«                         ║ 
« c13: 2/═════════════════╩═
«                         0

In [112]:
#Creating and running the job in the machine.
backend = Aer.get_backend("qasm_simulator")
qjob = execute(qc ,backend=backend ,shots =1024)
result = qjob.result ()
stats = result.get_counts ()
#Measure the winning percentage
won = 0
lost = 0
for dat in stats:
    if (int(dat [3])*int(dat [4]))==(( int(dat [0])+int(dat [1]))%2): # Check if the condition is satisfied.
        won += stats[dat]
    else:
        lost += stats[dat]
print("The number of games won : ", won)
print("The number of games lost : ", lost)

The number of games won :  881
The number of games lost :  143
